In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 22 kB/s 


In [ ]:
from typing import Optional, Sequence
import torch.nn as nn
class ClassifierMLP(nn.Module):
    def __init__(self, in_ch: int, num_classes: Optional[int], layers_description: Sequence[int]=(256,128), dropout_rate: float = 0.1):
        super().__init__()
        layer_list = []
        layer_list.append(nn.Linear(in_ch, layers_description[0]))
        layer_list.append(nn.ReLU())
        if dropout_rate is not None and dropout_rate > 0:
            layer_list.append(nn.Dropout(p=dropout_rate))
        last_layer_size = layers_description[0]
        for curr_layer_size in layers_description[1:]:
            layer_list.append(nn.Linear(last_layer_size, curr_layer_size))
            layer_list.append(nn.ReLU())
            if dropout_rate is not None and dropout_rate > 0:
                layer_list.append(nn.Dropout(p=dropout_rate))
            last_layer_size = curr_layer_size
        
        if num_classes is not None:
            layer_list.append(nn.Linear(last_layer_size, num_classes))
        
        self.classifier = nn.Sequential(*layer_list)

    def forward(self, x):
        x = self.classifier(x)
        return x
  
model=ClassifierMLP(12,2)  
import torch    

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pathmodel='/content/drive/MyDrive/knights_challenege/models/fold1mlptask1un/mlpmodelfold1.pth'
trainedmodel=torch.load(pathmodel)
model.load_state_dict(trainedmodel)
model.eval()
model.to(device)

ClassifierMLP(
  (classifier): Sequential(
    (0): Linear(in_features=12, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [ ]:
import pandas as pd
pathdata='/content/drive/MyDrive/knights_challenege/task1fold/test_un_moralize_fold1.csv'
datasetval=pd.read_csv(pathdata)

In [ ]:
#datasetval
datasetval.columns

Index(['SubjectId', 'age', 'bmi', 'gender', 'gender_num', 'comorbidities',
       'smoking_history', 'radiographic_size', 'preop_egfr', 'alcohol_use',
       'chewing_tobacco_use', 'x_spacing', 'y_spacing', 'z_spacing',
       'aua_risk_group', 'task_1_label', 'task_2_label'],
      dtype='object')

In [ ]:
#NoAT-score
#CanAT-score
#case_id
datatarget=datasetval['task_1_label'].copy()
import torch
#pathmodel='/content/drive/MyDrive/knight_challenegs2022/models/fold0model/3dmodeltask1.pth'
#trainedmodel=torch.load(pathmodel)
#model.load_state_dict(trainedmodel)
model.eval()
model.to(device)
dataframe={'case_id':[],
           'NoAT-score':[],
           'CanAT-score':[]}
CLINICAL_NAMES=['case_id','NoAT-score','CanAT-score']
CLINICAL_NAMES1=['case_id','Task1-target']
#feature=in_out.drop(['SubjectId','gender','aua_risk_group',
                          #'task_1_label','task_2_label'],axis=1, inplace=True)
import numpy as np
df = pd.DataFrame(columns=CLINICAL_NAMES)
df1=pd.DataFrame(columns=CLINICAL_NAMES1)
for i,patinet in enumerate(datasetval['SubjectId']):
  #pat=patinet.split('_')[-1]
  #pat=str(0000)+i
  number_str = str(i)
  pat = number_str.zfill(5)
  #print(i)
  #print(patinet)
  features=datasetval.iloc[i]
  target=datasetval['task_1_label'].iloc[i]
  feature=features.drop(['SubjectId','gender','aua_risk_group',
                          'task_1_label','task_2_label'])
  #feature1=feature.drop(['gender'])
  #print(feature1.shape)
  feature1=feature.copy()
  x_f_array=np.array(feature1).astype(float)
  #print(x_f_array)
  #.astype(int)
  x_f_array_t=torch.from_numpy(x_f_array).float()
  x_f_array_t=torch.unsqueeze(x_f_array_t,axis=0).to(device)
  prediction=model(x_f_array_t)
  output=torch.softmax(prediction, dim=1)
  output=torch.squeeze(output,axis=0)
  pred=output.detach().cpu().numpy()
  #print(output.detach().cpu().numpy())
  df.loc[i,'case_id']=pat
  df.loc[i,'NoAT-score']='%.1f' % pred[0]
  df.loc[i,'CanAT-score']='%.1f' % pred[1]
  df1.loc[i,'case_id']=pat
  df1.loc[i,'Task1-target']=target

  #dataframe['case_id'].append(str(pat))
  #dataframe['NoAT-score'].append(pred[0])
  #dataframe['CanAT-score'].append(pred[1])
  #features.iloc[i]
  #print(x_f_array_t.shape)
  #print(pat)

In [ ]:
df.to_csv('example_task1_predictions.csv',index=False)

In [ ]:
df1.to_csv('example_targets.csv',index=False)

In [ ]:
df['case_id'] = df['case_id'].apply('="{}"'.format)

In [ ]:
df.to_csv('task1_predictions.csv',index=False)